# Data Collection Scripts

In [55]:
import pandas as pd
from tabula.io import read_pdf
import matplotlib.pyplot as plt

## Kenyan Data Cleaning
Outpiting CSV files for Kenyan cleaned data as it was initially collected from [Kenya's Energy and Petroleum Regulation Authority (EPRA)](https://www.epra.go.ke/services/petroleum/petroleum-prices/).

In [25]:
kenya_df = pd.read_csv('kenya_consolidated.csv')
kenya_df.head()

,start_date,end_date,town,super_petrol,diesel,kerosene
0,15/07/2023,14/08/2023,Mombasa,191.62,176.63,166.43
1,15/07/2023,14/08/2023,Kilifi,192.39,177.40,167.19
2,15/07/2023,14/08/2023,Likoni Mainland,192.00,177.02,166.81
3,15/07/2023,14/08/2023,Kwale,192.00,177.02,166.81
4,15/07/2023,14/08/2023,Malindi,192.62,177.62,167.41


In [26]:
kenya_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3531 entries, 0 to 3530
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   start_date    3531 non-null   object 
 1   end_date      3531 non-null   object 
 2   town          3531 non-null   object 
 3   super_petrol  3531 non-null   float64
 4   diesel        3531 non-null   float64
 5   kerosene      3531 non-null   float64
dtypes: float64(3), object(3)
memory usage: 165.6+ KB


In [27]:
## See the unique start dates and end dates (They should be have same unique values)
start_dates = kenya_df.start_date.unique()
end_dates = kenya_df.end_date.unique()
print("Start Dates: ", start_dates,start_dates.shape, "\n End Dates: ", end_dates, end_dates.shape)

Start Dates:  ['15/07/2023' '15/06/2023' '15/04/2023' '15/01/2023' '15/12/2022'
 '15/11/2022' '15/10/2022' '15/09/2022' '15/08/2022' '15/07/2022'
 '15/06/2022' '15/05/2022' '15/04/2022' '15/03/2022' '15/02/2022'
 '15/01/2022' '15/11/2021'] (17,) 
 End Dates:  ['14/08/2023' '14/07/2023' '14/05/2023' '14/02/2023' '14/01/2023'
 '14/12/2022' '14/11/2022' '14/10/2022' '14/09/2022' '14/08/2022'
 '14/07/2022' '14/06/2022' '14/05/2022' '14/04/2022' '14/03/2022'
 '14/02/2022' '14/12/2021'] (17,)


In [41]:
# Separate data by start date and save data in different CSV files named by the date
for date in start_dates:
    df = kenya_df[kenya_df.start_date == date]
    fname = "kenya_"+ date[-4:] + "_" + date[3:5] + "_" + date[:2] + ".csv" #Format file name: kenya_year_month_day.csv
    df.to_csv(fname)

You can now access the Kenyan data files based on their dates (As available through EPRA), or you can access all the data available from EPRA through the file `kenya_consolidated.csv`.

## Tanzania Data Collection
Tanzanian data from the [Energy and Water Utilities Authority (EWURA)](https://www.ewura.go.tz/fuel-prices/) was only available in PDF format. Below we'll design scripts to seamlessly extract data from the PDF documents into CSV files.

In [56]:
link = "https://www.ewura.go.tz/wp-content/uploads/2023/10/Cap-Prices-wef-4th-October-2023-English-03102023-1.pdf"
dfs = read_pdf(link, pages='all')
dfs

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>